## Show tables with Scala and Spark



In [1]:
spark.catalog.listTables.show(false)

+--------------------------------+--------+-----------+---------+-----------+
|name                            |database|description|tableType|isTemporary|
+--------------------------------+--------+-----------+---------+-----------+
|btl_departures_arrivals_airports|default |null       |EXTERNAL |false      |
|btl_distances                   |default |null       |EXTERNAL |false      |
|int_airports                    |default |null       |EXTERNAL |false      |
|int_departures                  |default |null       |EXTERNAL |false      |
+--------------------------------+--------+-----------+---------+-----------+



## Use SQL

<div>To display result you need to click on the table symbol after "Out: DataFrame", then execute the new inspection cell...</div><div>For me this is a little bit troublesome in the current Polynote version. I like more using Spark DataFrame's show statement.</div>

In [3]:
select * from default.btl_distances where estarrivalairport = 'LEPA'

In [4]:
spark.sql("select * from default.btl_distances where estarrivalairport = 'LEPA'").show

+-------------------+-----------------+--------------------+----------------+-----------------+-----------------+----------------+-----------------+-----------------+---------------------+
|estdepartureairport|estarrivalairport|            arr_name|arr_latitude_deg|arr_longitude_deg|         dep_name|dep_latitude_deg|dep_longitude_deg|         distance|could_be_done_by_rail|
+-------------------+-----------------+--------------------+----------------+-----------------+-----------------+----------------+-----------------+-----------------+---------------------+
|               LSZB|             LEPA|Palma de Mallorca...|       39.551701|          2.73881|Bern-Belp Airport|       46.913419|         7.499747|904.4677807313127|                false|
+-------------------+-----------------+--------------------+----------------+-----------------+-----------------+----------------+-----------------+-----------------+---------------------+



In [5]:
spark.table("default.int_departures").where($"estarrivalairport"==="LEPA").show

+-----------------------------+--------+-------------------------------+-----------------+------------------------------+-----------------------------+-------------------+--------------------------------+-------------------------------+----------+------+----------+--------+--------------------+
|arrivalAirportCandidatesCount|callsign|departureAirportCandidatesCount|estArrivalAirport|estArrivalAirportHorizDistance|estArrivalAirportVertDistance|estDepartureAirport|estDepartureAirportHorizDistance|estDepartureAirportVertDistance| firstSeen|icao24|  lastSeen|      dt|      dl_ts_captured|
+-----------------------------+--------+-------------------------------+-----------------+------------------------------+-----------------------------+-------------------+--------------------------------+-------------------------------+----------+------+----------+--------+--------------------+
|                            1|OAW24H  |                              0|             LEPA|                      

## Select data by using DataObjects configured in SmartDataLake



In [9]:
import io.smartdatalake.generated.DataObjectCatalog
import io.smartdatalake.lab.SmartDataLakeBuilderLab
val sdlb = SmartDataLakeBuilderLab[DataObjectCatalog](spark, Seq("/mnt/config"), DataObjectCatalog(_,_), Some(getClass.getClassLoader))
val data = sdlb.data
implicit val context = sdlb.context
implicit val ss = spark

In [11]:
// get a dataobject
val dataIntAirports = data.intAirports.get.show
val dataIntDepartures = data.intDepartures.get.show

+-----+--------------------+------------------+-------------------+--------------------+-------------------+
|ident|                name|      latitude_deg|      longitude_deg|      dl_ts_captured|    dl_ts_delimited|
+-----+--------------------+------------------+-------------------+--------------------+-------------------+
|  00A|   Total Rf Heliport|    40.07080078125| -74.93360137939453|2023-03-12 11:26:...|9999-12-31 00:00:00|
| 00AA|Aero B Ranch Airport|         38.704022|        -101.473911|2023-03-12 11:26:...|9999-12-31 00:00:00|
| 00AK|        Lowell Field|         59.947733|        -151.692524|2023-03-12 11:26:...|9999-12-31 00:00:00|
| 00AL|        Epps Airpark| 34.86479949951172| -86.77030181884766|2023-03-12 11:26:...|9999-12-31 00:00:00|
| 00AS|      Fulton Airport|        34.9428028|        -97.8180194|2023-03-12 11:26:...|9999-12-31 00:00:00|
| 00AZ|      Cordes Airport|34.305599212646484|-112.16500091552734|2023-03-12 11:26:...|9999-12-31 00:00:00|
| 00CL| Williams Ag

In [12]:
//data.intAirports.dataObject.dropTable